# Задание 1. Анализ диалогов с Gradio

## 1.Работа с данными

### Загружаем данные

In [1]:
!gdown 14AtzLN5CyjsfU0XXnvCNs-ec4YGtvcHc

Downloading...
From: https://drive.google.com/uc?id=14AtzLN5CyjsfU0XXnvCNs-ec4YGtvcHc
To: /content/data.zip
100% 9.13M/9.13M [00:00<00:00, 39.4MB/s]


In [2]:
!unzip data.zip -d data

Archive:  data.zip
  inflating: data/movie_characters_metadata.tsv  
  inflating: data/movie_conversations.tsv  
  inflating: data/movie_lines.tsv    
  inflating: data/movie_titles_metadata.tsv  


### Импортируем библиотеки

In [3]:
import pandas as pd
import ast
import torch
from tqdm import tqdm
pd.options.mode.chained_assignment = None

### Читаем данные

In [85]:
characters = pd.read_csv('/content/data/movie_characters_metadata.tsv',
                         sep='\t',
                         warn_bad_lines=False,
                         error_bad_lines=False,
                         header=None
                         )
conversations = pd.read_csv('/content/data/movie_conversations.tsv',
                            sep='\t',
                            warn_bad_lines=False,
                            error_bad_lines=False,
                            header=None
                            )

lines = pd.read_csv('/content/data/movie_lines.tsv',
                    sep='\t',
                    warn_bad_lines=False,
                    error_bad_lines=False,
                    header=None
                    )
titles = pd.read_csv('/content/data/movie_titles_metadata.tsv',
                     sep='\t',
                     warn_bad_lines=False,
                     error_bad_lines=False,
                     header=None
                     )

<ipython-input-85-629c45eb9c6f>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  characters = pd.read_csv('/content/data/movie_characters_metadata.tsv',
<ipython-input-85-629c45eb9c6f>:1: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  characters = pd.read_csv('/content/data/movie_characters_metadata.tsv',
<ipython-input-85-629c45eb9c6f>:7: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  conversations = pd.read_csv('/content/data/movie_conversations.tsv',
<ipython-input-85-629c45eb9c6f>:7: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  conversations = pd.read_csv('/content/data/movie_conversations.tsv',
<ipython-input-8

In [5]:
characters.head(1)

,0,1,2,3,4,5
0,u0,BIANCA,m0,10 things i hate about you,f,4


In [6]:
conversations.head(1)

,0,1,2,3
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']


In [9]:
lines.head(1)

,0,1,2,3,4
0,L1045,u0,m0,BIANCA,They do not!


In [10]:
titles.head(1)

,0,1,2,3,4,5
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']


### Задаем названия

In [86]:
characters.columns = ['character_id', 'character', 'movie_id', 'movie_title',
                      'gender',
                      'position']
conversations.columns = ['character_id_1', 'character_id_2', 'movie_id', 'order']
lines.columns = ['line_id', 'character_id', 'movie_id', 'character', 'text']
titles.columns = ['movie_id', 'movie_title', 'movie_year', 'rating', 'imdb_votes', 'genres']

characters.dropna(inplace=True)
conversations.dropna(inplace=True)
lines.dropna(inplace=True)
titles.dropna(inplace=True)

In [12]:
characters.head()

,character_id,character,movie_id,movie_title,gender,position
0,u0,BIANCA,m0,10 things i hate about you,f,4
1,u1,BRUCE,m0,10 things i hate about you,?,?
2,u2,CAMERON,m0,10 things i hate about you,m,3
3,u3,CHASTITY,m0,10 things i hate about you,?,?
4,u4,JOEY,m0,10 things i hate about you,m,6


In [13]:
conversations.head()

,character_id_1,character_id_2,movie_id,order
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']


In [13]:
lines.head()

,line_id,character_id,movie_id,character,text
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [14]:
titles.head()

,movie_id,movie_title,movie_year,rating,imdb_votes,genres
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']


### Задаем необходимые функции для преобразования данных

In [87]:
# Функция для преобразования колонки 'order'(порядок реплик)
# возвращает список строк после разделения строки s на основе разделителя ' '.
def ordr(s, delim=' '):
    s1 = ast.literal_eval(s)
    to_ret = [delim + i for i in s1[0].split(delim)[1:]]
    return to_ret

In [88]:
# Функция для преобразования колонки 'genres'
# удаляет '][' из строки и разделяет ее на список строк.
def gnr(s):
    to_ret = [i[1:-1] for i in s.strip('][').split(' ')]
    return to_ret

### Преобразуем и объединяем данные

In [89]:
lns_chrs = dict(zip(lines.line_id, lines.character_id)) # Маппинг персонажей
chars = dict(zip(characters.character_id, characters.character)) # Маппинг имен
gndr = dict(zip(characters.character_id, characters.gender)) # Маппинг пола
lns = dict(zip(lines.line_id, lines.text)) # Маппинг текста

In [90]:
titles['genres'] = titles['genres'].apply(lambda x: gnr(x))
titles_new = titles[['movie_id', 'movie_title', 'genres']]

conversations['order'] = conversations['order'].apply(lambda x: ordr(x, delim='L'))
conversations['characters'] = conversations['order'].apply(lambda x: [lns_chrs[i] if i \
                                                                       in lns_chrs.keys() else 'unknown_character' for i in x])

conversations['genders'] = conversations['characters'].apply(lambda x: [gndr[i] if i \
                                                                        in gndr.keys() else '?' for i in x])

conversations['dialog'] = conversations['order'].apply(lambda x: [lns[i] if i \
                                                                    in lns.keys() else 'unknown_text'for i in x])

conversations['names'] = conversations['characters'].apply(lambda x: [chars[i] if i \
                                                                      in chars.keys() else 'unknown_character 'for i in x])

conversations_new = conversations[['movie_id', 'order', 'characters',
                                   'genders', 'dialog', 'names']]



df = conversations_new.merge(titles_new, on='movie_id', how='left')

In [91]:
# получившийся датафрейм
df

,movie_id,order,characters,genders,dialog,names,movie_title,genres
0,m0,"[L194, L195, L196, L197]","[u0, u2, u0, u2]","[f, m, f, m]",[Can we make this quick? Roxanne Korrine and ...,"[BIANCA, CAMERON, BIANCA, CAMERON]",10 things i hate about you,"[comedy, romance]"
1,m0,"[L198, L199]","[u0, u2]","[f, m]",[You're asking me out. That's so cute. What's...,"[BIANCA, CAMERON]",10 things i hate about you,"[comedy, romance]"
2,m0,"[L200, L201, L202, L203]","[u0, u2, u0, u2]","[f, m, f, m]",[No no it's my fault -- we didn't have a prope...,"[BIANCA, CAMERON, BIANCA, CAMERON]",10 things i hate about you,"[comedy, romance]"
3,m0,"[L204, L205, L206]","[u2, u0, u2]","[m, f, m]","[Why?, Unsolved mystery. She used to be reall...","[CAMERON, BIANCA, CAMERON]",10 things i hate about you,"[comedy, romance]"
4,m0,"[L207, L208]","[u0, u2]","[f, m]",[Gosh if only we could find Kat a boyfriend......,"[BIANCA, CAMERON]",10 things i hate about you,"[comedy, romance]"
...,...,...,...,...,...,...,...,...
83092,m616,"[L666324, L666325, L666326, L666327]","[u9028, u9031, u9028, u9031]","[?, ?, ?, ?]",[Do you think she might be interested in some...,"[COGHILL, MELVILL, COGHILL, MELVILL]",zulu dawn,"[action, adventure, drama, history, war]"
83093,m616,"[L666575, L666576]","[u9028, u9031]","[?, ?]",[Choose your targets men. That's right Watch t...,"[COGHILL, MELVILL]",zulu dawn,"[action, adventure, drama, history, war]"
83094,m616,"[L666256, L666257]","[u9034, u9030]","[?, ?]",[Colonel Durnford... William Vereker. I hear y...,"[VEREKER, DURNFORD]",zulu dawn,"[action, adventure, drama, history, war]"
83095,m616,"[L666369, L666370, L666371, L666372]","[u9030, u9034, u9030, u9034]","[?, ?, ?, ?]","[Your orders Mr Vereker?, I'm to take the Sika...","[DURNFORD, VEREKER, DURNFORD, VEREKER]",zulu dawn,"[action, adventure, drama, history, war]"


### Cохраняем датафрейм

In [92]:
df.to_csv('/content/data/movies_prepared.csv', index=False)

## 2.Sentiment

In [ ]:
!pip install transformers
!pip install ray[serve]
!pip install fastapi
!pip install xformers
!pip install emoji

In [36]:
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
import ray
from ray import serve
from fastapi import FastAPI
from pydantic import PositiveInt, constr, BaseModel
import requests
from typing import List

In [93]:
df = pd.read_csv('/content/data/movies_prepared.csv')
df.head()

,movie_id,order,characters,genders,dialog,names,movie_title,genres
0,m0,"['L194', 'L195', 'L196', 'L197']","['u0', 'u2', 'u0', 'u2']","['f', 'm', 'f', 'm']",['Can we make this quick? Roxanne Korrine and...,"['BIANCA', 'CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
1,m0,"['L198', 'L199']","['u0', 'u2']","['f', 'm']","[""You're asking me out. That's so cute. What'...","['BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
2,m0,"['L200', 'L201', 'L202', 'L203']","['u0', 'u2', 'u0', 'u2']","['f', 'm', 'f', 'm']","[""No no it's my fault -- we didn't have a prop...","['BIANCA', 'CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
3,m0,"['L204', 'L205', 'L206']","['u2', 'u0', 'u2']","['m', 'f', 'm']","['Why?', 'Unsolved mystery. She used to be re...","['CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
4,m0,"['L207', 'L208']","['u0', 'u2']","['f', 'm']",['Gosh if only we could find Kat a boyfriend.....,"['BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"


In [94]:
df['dialog'] = df['dialog'].apply(lambda x: eval(x))
df['characters'] = df['characters'].apply(lambda x: eval(x))
df.head()

,movie_id,order,characters,genders,dialog,names,movie_title,genres
0,m0,"['L194', 'L195', 'L196', 'L197']","[u0, u2, u0, u2]","['f', 'm', 'f', 'm']",[Can we make this quick? Roxanne Korrine and ...,"['BIANCA', 'CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
1,m0,"['L198', 'L199']","[u0, u2]","['f', 'm']",[You're asking me out. That's so cute. What's...,"['BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
2,m0,"['L200', 'L201', 'L202', 'L203']","[u0, u2, u0, u2]","['f', 'm', 'f', 'm']",[No no it's my fault -- we didn't have a prope...,"['BIANCA', 'CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
3,m0,"['L204', 'L205', 'L206']","[u2, u0, u2]","['m', 'f', 'm']","[Why?, Unsolved mystery. She used to be reall...","['CAMERON', 'BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"
4,m0,"['L207', 'L208']","[u0, u2]","['f', 'm']",[Gosh if only we could find Kat a boyfriend......,"['BIANCA', 'CAMERON']",10 things i hate about you,"['comedy', 'romance']"


In [ ]:
ray.init()

In [32]:
serve.start()

2023-06-17 06:31:58,163	INFO worker.py:1627 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


(HTTPProxyActor pid=4180) INFO:     Started server process [4180]


### API

In [205]:
class Row(BaseModel):
    dialog: List[str]
    characters: List[str]
    movie_name: str
    genres: List[str]

app = FastAPI()

@serve.deployment
@serve.ingress(app)
class SentimentAnalysis:
    def __init__(self):
        self.classifier = pipeline("sentiment-analysis",
                                   model="yiyanghkust/finbert-tone")

    @app.get("/text")
    def get_sentiment_text(
        self,
        text: constr(min_length=1, strip_whitespace=True),
        max_length: PositiveInt = 256,
    ):
        if len(text) > max_length:
          text = text[:max_length]
        sentiment_list = self.classifier(text)
        sentiment = sentiment_list[0]["label"]
        return sentiment

    @app.get("/batch")
    @serve.batch(max_batch_size=10, batch_wait_timeout_s=0.1)
    async def classify_batched(self, batched_inputs):
        print("Got batch size:", len(batched_inputs))
        sentiments = self.classifier(batched_inputs)
        return [sentiment["label"] for sentiment in sentiments]

    @app.post("/dataframe/")
    def get_sentiment_text(
        self,
        df_param: List[Row],
        max_replics: PositiveInt = 10,
    ):
        if len(df_param) > max_replics:
          df_param = df_param[:max_replics]
        sentiment_list = self.classifier([r.dialog for r in df_param][0])
        sentiment = [r['label'] for r in sentiment_list]
        return sentiment


SentimentAnalysis.deploy()

(ServeController pid=4140) INFO 2023-06-17 08:44:04,687 controller 4140 deployment_state.py:1298 - Deploying new version of deployment SentimentAnalysis.
(ServeController pid=4140) INFO 2023-06-17 08:44:04,760 controller 4140 deployment_state.py:1425 - Stopping 1 replicas of deployment 'SentimentAnalysis' with outdated versions.
(ServeController pid=4140) INFO 2023-06-17 08:44:06,923 controller 4140 deployment_state.py:1537 - Adding 1 replica to deployment SentimentAnalysis.
(ServeReplica:SentimentAnalysis pid=38228) 2023-06-17 08:44:10.835359: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Запрос /text

In [206]:
response = requests.get(
    "http://127.0.0.1:8000/SentimentAnalysis/text", params={"text":'I hate people'}
).text

response

'"Negative"'

Запрос /batch

In [207]:
handleSimple = serve.get_deployment('SentimentAnalysis').get_handle()
ray.get([handleSimple.classify_batched.remote("I hate people") for _ in range(10)])

(ServeReplica:SentimentAnalysis pid=38228) Got batch size: 10


['Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative',
 'Negative']

Запрос /dataframe/

In [208]:
response = requests.post(
    "http://127.0.0.1:8000/SentimentAnalysis/dataframe/", json=[{"dialog": ["Did you know that you are crazy?"],
                                                                   "genres": ["fantastic"],
                                                                   "movie_name": "Green Elephant",
                                                                   "characters": ["Epifancev"]}]
).text

response

'["Negative"]'

### Интерфейс

In [ ]:
!pip install gradio

In [17]:
from ray.serve.gradio_integrations import GradioServer
import gradio as gr
import matplotlib.pyplot as plt
from ray import serve
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [217]:
classifier = pipeline("sentiment-analysis",
                  model="yiyanghkust/finbert-tone")

example_input = (
    {'dialog': ['I hate people', 'I love you', 'My name is Vika', 'Hello Vika!'],
     'characters': ['Petya', 'Vasya', 'Vika', 'Petya']}
)


def pie_chart(resp):
    labels = ["Positive", "Negative", "Neutral"]
    colors = ['lightgreen', 'red','royalblue']

    fig = make_subplots(rows=1, cols=len(resp),
                        specs=[[{'type': 'domain'} for _ in range(len(resp))]],
                        subplot_titles=[key for key in resp.keys()])
    for i, p in enumerate(resp.keys(), 1):

      fig.add_trace(go.Pie(labels=labels,
                           values=[value for value in resp[p].values()],), 1, i)

    fig.update_traces(hoverinfo="label+percent", textfont_size=20,
                      marker=dict(colors=colors, line=dict(color='#000000', width=1)))
    fig.update_layout(title_text='Emotions in dialog')

    return fig

def classify(str_data):
    data = eval(str_data)
    return classifier(data)


def count_sentiment_per_person(str_dict_data):
    dict_data = eval(str_dict_data)
    labels = [pred['label'] for pred in classifier(dict_data['dialog'])]
    resp = {}
    for i in range(len(labels)):
      person = dict_data['characters'][i]
      label = labels[i]
      if not(person in resp):
        resp[person] = {'Positive': 0, 'Negative': 0, 'Neutral': 0}
      if not(label in resp[person]):
        resp[person][label] = 0
      resp[person][label] += 1

    return pie_chart(resp)


def gradio_SentimentAnalysis_builder():
    return gr.Interface(
        fn=count_sentiment_per_person,
        inputs=[gr.Textbox(value=example_input, label="Окно ввода")],
        outputs=[gr.Plot()],
    )

In [218]:
gradio_SentimentAnalysis_builder().launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [220]:
serve.run(
    GradioServer.options(ray_actor_options={"num_cpus": 1}).bind(
        gradio_SentimentAnalysis_builder
    )
)

## 3.Genre + Emotions

In [18]:
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm
import gradio as gr

In [19]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, Sampler, TensorDataset

In [113]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

In [ ]:
model.to('cuda:0')

In [258]:
df = pd.read_csv('/content/data/movies_prepared.csv')

Преобразуем строки в списки

In [259]:
df.dialog = df.dialog.apply(lambda x: eval(x))
df.order = df.order.apply(lambda x: eval(x))
df.characters = df.characters.apply(lambda x: eval(x))
df.genders = df.genders.apply(lambda x: eval(x))
df.genders = df.genders.apply(lambda x: x if '?' not in x else np.nan)

Cоздаем новый ДФ в который помещаем диалоги по длине < 5

In [ ]:
df['dialog_len'] = df.dialog.apply(lambda x: len(x))
df_new = df[df.dialog_len < 5]
df_new.dropna(inplace=True)
df_new = df_new.sample(50, random_state=18)
df_new.reset_index(inplace=True, drop=True)

In [146]:
df_new.head()

,movie_id,order,characters,genders,dialog,names,movie_title,genres,dialog_len
0,m615,"[L665758, L665759, L665760, L665761]","[u9017, u9019, u9017, u9019]","[M, m, M, m]",[The ancient masters promised impossibilities ...,"['FREDDY', 'IGOR', 'FREDDY', 'IGOR']",young frankenstein,"['comedy', 'sci-fi']",4
1,m188,"[L522640, L522641]","[u2852, u2860]","[f, m]","[Tommy...!, I'm all right.]","['ELLIE', 'TOMMY']",someone to watch over me,"['action', 'crime', 'drama', 'romance', 'thril...",2
2,m401,"[L309132, L309133]","[u6033, u6035]","[m, f]",[I don't think so Thea. He has spoken of your ...,"['OLIVER', 'THEA']",isle of the dead,"['drama', 'horror', 'mystery', 'thriller']",2
3,m570,"[L609752, L609753, L609754, L609755]","[u8410, u8409, u8410, u8409]","[f, M, f, M]",[We're American soldiers. We don't abandon pe...,"['WES', 'JAEGER', 'WES', 'JAEGER']",three kings,"['action', 'adventure', 'comedy', 'drama', 'war']",4
4,m235,"[L15348, L15349]","[u3572, u3574]","[m, m]",[Eh. I thought you were looking after the old ...,"['PAUL', 'RINGO']",a hard day's night,"['comedy', 'music']",2


### Размечаем данные

In [147]:
def convert_to_dataset_torch(data: str):
    input_ids = []
    attention_masks = []

    encoded_dict = tokenizer.encode_plus(data,
                                             max_length=130,
                                             pad_to_max_length=True,
                                             return_attention_mask=True,
                                             return_tensors='pt',
                                             truncation=True)

    input_ids.append(encoded_dict['input_ids'])

    attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    input_ids.to(dtype=torch.long)
    attention_masks.to(dtype=torch.long)

    return input_ids, attention_masks

In [119]:
labels = [
    'Positive',
    'Negative',
    'Neutral'
]
mapping = {i: k for i, k in enumerate(labels)}

In [149]:
preds_all = []
for i in tqdm(df_new.dialog):
    preds_dial = []
    for k in i:
        converted = convert_to_dataset_torch(k)
        preds = model(input_ids=converted[0].to('cuda:0'),
                      attention_mask=converted[1].to('cuda:0')).logits
        predictions = torch.argmax(torch.nn.Softmax()(preds), axis=1)
        preds_dial.append(predictions.item())
    preds_all.append(preds_dial)

  0%|          | 0/50 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
<ipython-input-149-2aaf0c17391b>:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  predictions = torch.argmax(torch.nn.Softmax()(preds), axis=1)
100%|██████████| 50/50 [00:05<00:00,  9.66it/s]


In [150]:
df_new['predictions'] = pd.Series(preds_all)

In [151]:
df_new.head()

,movie_id,order,characters,genders,dialog,names,movie_title,genres,dialog_len,predictions
0,m615,"[L665758, L665759, L665760, L665761]","[u9017, u9019, u9017, u9019]","[M, m, M, m]",[The ancient masters promised impossibilities ...,"['FREDDY', 'IGOR', 'FREDDY', 'IGOR']",young frankenstein,"['comedy', 'sci-fi']",4,"[1, 1, 2, 2]"
1,m188,"[L522640, L522641]","[u2852, u2860]","[f, m]","[Tommy...!, I'm all right.]","['ELLIE', 'TOMMY']",someone to watch over me,"['action', 'crime', 'drama', 'romance', 'thril...",2,"[1, 2]"
2,m401,"[L309132, L309133]","[u6033, u6035]","[m, f]",[I don't think so Thea. He has spoken of your ...,"['OLIVER', 'THEA']",isle of the dead,"['drama', 'horror', 'mystery', 'thriller']",2,"[1, 0]"
3,m570,"[L609752, L609753, L609754, L609755]","[u8410, u8409, u8410, u8409]","[f, M, f, M]",[We're American soldiers. We don't abandon pe...,"['WES', 'JAEGER', 'WES', 'JAEGER']",three kings,"['action', 'adventure', 'comedy', 'drama', 'war']",4,"[2, 1, 0, 1]"
4,m235,"[L15348, L15349]","[u3572, u3574]","[m, m]",[Eh. I thought you were looking after the old ...,"['PAUL', 'RINGO']",a hard day's night,"['comedy', 'music']",2,"[0, 1]"


Сохраняем размеченный фрейм

In [152]:
df_new.to_csv('/content/data/movies_labeled.csv', index=False)

In [155]:
df = pd.read_csv('/content/data/movies_labeled.csv')
df.dialog = df.dialog.apply(lambda x: eval(x))
df.order = df.order.apply(lambda x: eval(x))
df.characters = df.characters.apply(lambda x: eval(x))
df.genders = df.genders.apply(lambda x: eval(x))
df.genres = df.genres.apply(lambda x: eval(x))
df.predictions = df.predictions.apply(lambda x: eval(x))
df.genders = df.genders.apply(lambda x: ['Мужчина' if i in ['m', 'M'] else 'Женщина' for i in x])

In [156]:
df.predictions = df.predictions.apply(lambda x: [*map(mapping.get, x)])

In [158]:
available_genres = df.genres.explode().unique().tolist()

In [159]:
available_genres

['comedy',
 'sci-fi',
 'action',
 'crime',
 'drama',
 'romance',
 'thriller',
 'horror',
 'mystery',
 'adventure',
 'war',
 'music',
 'biography',
 'fantasy',
 'western',
 'animation',
 'family']

### Интерфейс

In [248]:
import plotly.express as px
import plotly.graph_objects as go

def genres_stat_plotly(genre):
    df['need_genre'] = df.genres.apply(lambda x: True if genre in x else False)
    df_final = df.query('need_genre == True')
    df_final.reset_index(inplace=True, drop=True)
    df_final = df_final[['characters', 'genders', 'predictions']]
    df_final = df_final.explode(['characters', 'genders', 'predictions'])
    df_final = df_final.groupby('genders')['predictions'].value_counts(normalize=True)


    fig = make_subplots(rows=1, cols=2,
                        specs=[[{'type': 'domain'} for _ in range(2)]],
                        subplot_titles=['Женщины','Мужчины'])
    i = 1
    for x in df_final.index.get_level_values('genders').unique():
        fig.add_trace(go.Pie(values=df_final[x], labels=df_final[x].index, name=x), 1, i)
        i += 1
    fig.update_traces(hoverinfo="label+percent", textfont_size=20)
    fig.update_layout(title='Статистика для жанра {}'.format(genre))  # add title to the figure

    return fig

In [254]:
theme = gr.themes.Monochrome(primary_hue="blue", secondary_hue="blue").set(
    button_primary_background_fill="#FF0000",
    button_primary_background_fill_hover="*button_primary_background_fill",
)

In [255]:
%%capture
with gr.Blocks(theme=theme) as app:
    with gr.Row():
        gen = gr.Dropdown(available_genres, label='Жанр')
        btn = gr.Button(value="Получить статистику по эмоциям")
    with gr.Row():
        with gr.Column(scale=1, min_width=400):
            btn.click(genres_stat_plotly, inputs=gen, outputs=gr.Plot(label='Статистика'))

In [256]:
app.launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>